# 多类逻辑回归 --- 使用Gluon

现在让我们使用gluon来更快速地实现一个多类逻辑回归。

## 获取和读取数据

我们仍然使用FashionMNIST。我们将代码保存在[../utils.py](../utils.py)这样这里不用复制一遍。

In [3]:
!pip install mxnet

  Using cached mxnet-1.1.0-py2.py3-none-manylinux1_x86_64.whl
    100% |████████████████████████████████| 16.9MB 73kB/s  eta 0:00:01
  Using cached graphviz-0.8.1-py2.py3-none-any.whl
  Found existing installation: numpy 1.14.1
    Uninstalling numpy-1.14.1:
      Successfully uninstalled numpy-1.14.1
  Found existing installation: graphviz 0.8.2
    Uninstalling graphviz-0.8.2:
      Successfully uninstalled graphviz-0.8.2


In [4]:
import sys
sys.path.append('..')
import utils

batch_size = 256
train_data, test_data = utils.load_data_fashion_mnist(batch_size)

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL
/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/matplotlib/style/core.py:203: UserWarning: In /gpfs/fs01/user/s05b-0294eb3de63d2e-b5498f932ea5/.config/matplotlib/stylelib/my_custom_style.mplstyle: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(message)
/gpfs/fs01/user/s05b-0294eb3de63d2e-b5498f932ea5/.local/lib/python3.5/site

In [5]:
!ls

apache_access_log-beaa3.PROJECT  models			shakespeare.txt
brain_body_data.csv		 moviedataset.zip	simple-examples.tgz
data.npy			 mtcars (1).csv		summary_logs
datasets			 mtcars.csv		tf_logs
dl				 PierceCricketData.csv	utils2.py
kdd.gz				 ptb.zip		utils.py
langmod				 __pycache__		utils.pyc
lena (2).png			 reader.pyc		vocab.pkl
lena.png			 recsys
MNIST_data			 shakespeare (1).txt


## 定义和初始化模型

我们先使用Flatten层将输入数据转成 `batch_size` x `?` 的矩阵，然后输入到10个输出节点的全连接层。照例我们不需要制定每层输入的大小，gluon会做自动推导。

In [7]:
from mxnet import gluon

net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(10))
net.initialize()

## Softmax和交叉熵损失函数

如果你做了上一章的练习，那么你可能意识到了分开定义Softmax和交叉熵会有数值不稳定性。因此gluon提供一个将这两个函数合起来的数值更稳定的版本

In [8]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

## 优化

In [9]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

## 训练

In [10]:
from mxnet import ndarray as nd
from mxnet import autograd

for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)

    test_acc = utils.evaluate_accuracy(test_data, net)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data), train_acc/len(train_data), test_acc))

Epoch 0. Loss: 0.800408, Train acc 0.740318, Test acc 0.803686
Epoch 1. Loss: 0.576033, Train acc 0.810113, Test acc 0.816306
Epoch 2. Loss: 0.529953, Train acc 0.822666, Test acc 0.828425
Epoch 3. Loss: 0.507838, Train acc 0.828509, Test acc 0.838542
Epoch 4. Loss: 0.491577, Train acc 0.833400, Test acc 0.839343


## 结论

Gluon提供的函数有时候比手工写的数值更稳定。

## 练习

- 再尝试调大下学习率看看？
- 为什么参数都差不多，但gluon版本比从0开始的版本精度更高？

**吐槽和讨论欢迎点**[这里](https://discuss.gluon.ai/t/topic/740)